<a href="https://colab.research.google.com/github/helena-varela/Pipeline-Telemetria/blob/main/pipeline_telemetria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importação**

In [1]:
import json

**Salvar no txt**

In [2]:
def salvar_txt(dados):
  with open('telemetria.txt', 'a') as file:
    file.write(dados)

**Coleta de dados**

In [20]:
print("Bem-vindo")
id = input("Qual o ID do paciente? ")
pressao = input("Qual a pressão do paciente? ")
saturacao = input("Qual a saturação do paciente? ")
temperatura = input("Qual a temperatura do paciente? ")

dados = f"ID:{id}|pressao:{pressao}|saturacao:{saturacao}|temperatura:{temperatura}\n"

salvar_txt(dados)

Bem-vindo
Qual o ID do paciente? P005
Qual a pressão do paciente? invalida
Qual a saturação do paciente? 97
Qual a temperatura do paciente? 37.0


# **01. Extração e Limpeza (Extract & Transform)**

**Extraindo os dados**

In [22]:
# Extraindo ID
def extrai_ID(arquivo):
  lista_de_ids = []
  with open(arquivo, 'r') as file:
    for line in file:
      line_limpa = line.strip() # Retira os espaços vazios
      lista_partes = line_limpa.split('|') # Separa pelo |
      id = lista_partes[0] # O ID sempre será o primeiro elemento da lista
      id_e_numeros = id.split(':') # Separa pelo : ["ID", "P001"]
      id_paciente = id_e_numeros[1] # Pega apenas a parte do ID P001
      lista_de_ids.append(id_paciente) # Adiciona a lista

  return lista_de_ids

extrai_ID('telemetria.txt')

['P013', 'P001', 'P002', 'P003', 'P004', 'P001', 'P005']

In [26]:
# Extraindo Pressão Sistólica e Diastólica
def extrai_sistolica_diastolica(arquivo):
  lista_sistolica = []
  lista_diastolica = []
  with open(arquivo, 'r') as file:
    for line in file:
        line_limpa = line.strip() # Retira os espaços vazios
        lista_partes = line_limpa.split('|') # Separa pelo |
        try:
            pressoes = lista_partes[1]
            pressao_e_valores = pressoes.split(':')
            sis_e_dias = pressao_e_valores[1]
            separa_sis_dias = sis_e_dias.split('/')
            sistolica = separa_sis_dias[0]
            diastolica = separa_sis_dias[1]
            lista_sistolica.append(sistolica)
            lista_diastolica.append(diastolica)
        except (IndexError, ValueError):
            pass

  return lista_sistolica, lista_diastolica

extrai_sistolica_diastolica('telemetria.txt')

(['127', '185', '130', '140', '120', '190'],
 ['80', '95', '80', '90', '75', '100'])

In [25]:
# Extraindo Saturação
def extrai_saturacao(arquivo):
  lista_saturacao = []
  with open(arquivo, 'r') as file:
      for line in file:
        line_limpa = line.strip() # Retira os espaços vazios
        lista_partes = line_limpa.split('|') # Separa pelo |
        saturacao_parte = lista_partes[2]
        saturacao_separa = saturacao_parte.split(':')
        saturacao = saturacao_separa[1]
        lista_saturacao.append(saturacao)

  return lista_saturacao

extrai_saturacao('telemetria.txt')

['83', '89', '98', '95', '99', '91', '97']

In [27]:
# Extração da Temperatura
def extrai_temperatura(arquivo):
  lista_temperatura = []
  with open(arquivo, 'r') as file:
        for line in file:
          line_limpa = line.strip() # Retira os espaços vazios
          lista_partes = line_limpa.split('|') # Separa pelo |
          temperatura_parte = lista_partes[3]
          temperatura_separa = temperatura_parte.split(':')
          temperatura = temperatura_separa[1]
          lista_temperatura.append(temperatura)

  return lista_temperatura

extrai_temperatura('telemetria.txt')

['40', '38.1', 'trinta e sete', '', '36.8', '38.5', '37.0']

**Validação de Dados**

In [37]:
# Valida se algum campo está vazio
nome_arquivo_original = 'telemetria.txt'

linhas_completas = []
linhas_incompletas = []

try:
    with open(nome_arquivo_original, 'r') as file:
        for linha in file:
            linha_limpa = linha.strip()
            partes = linha_limpa.split('|')

            esta_completa = True

            for parte in partes:
                # Se uma parte não tiver ':' ou terminar com ':', o valor está faltando
                if ':' not in parte or parte.endswith(':'):
                    esta_completa = False
                    break

            if esta_completa:
                linhas_completas.append(linha_limpa)
            else:
                linhas_incompletas.append(linha_limpa)

    print("Linhas completas:")
    for linha in linhas_completas:
        print(linha)

    print("\nLinhas incompletas:")
    for linha in linhas_incompletas:
        print(linha)
        with open('log_errors.txt', 'a') as file:
          file.write(f"{linha}|Existe um campo vazio\n")

except FileNotFoundError:
    print(f"Erro: O arquivo '{nome_arquivo_original}' não foi encontrado.")

Linhas completas:
ID:P013|pressao:127/80|saturacao:83|temperatura:40
ID:P001|pressao:185/95|saturacao:89|temperatura:38.1
ID:P002|pressao:130/80|saturacao:98|temperatura:trinta e sete
ID:P004|pressao:120/75|saturacao:99|temperatura:36.8
ID:P001|pressao:190/100|saturacao:91|temperatura:38.5
ID:P005|pressao:invalida|saturacao:97|temperatura:37.0

Linhas incompletas:
ID:P003|pressao:140/90|saturacao:95|temperatura:


In [38]:
# Validação das Entradas
linhas_totalmente_validas = []

with open('log_errors.txt', 'a') as log_file:
    for linha in linhas_completas:
        partes = linha.strip().split('|')

        linha_e_valida = True

        # Validação da Pressão
        try:
            pressao_parte = partes[1]
            valor_pressao_texto = pressao_parte.split(':')[1]
            sistolica_str, diastolica_str = valor_pressao_texto.split('/')
            int(sistolica_str)
            int(diastolica_str)
        except (ValueError, IndexError):
            log_file.write(f"{linha.strip()} | O valor da pressão não é um inteiro\n")
            linha_e_valida = False

        # Validação da Saturação
        if linha_e_valida:
            try:
                saturacao_parte = partes[2]
                valor_saturacao = saturacao_parte.split(':')[1]
                int(valor_saturacao)
            except (ValueError, IndexError):
                log_file.write(f"{linha.strip()} | O valor da saturação não é um inteiro\n")
                linha_e_valida = False

        # Validação da Temperatura
        if linha_e_valida:
            try:
                temperatura_parte = partes[3]
                valor_temperatura = temperatura_parte.split(':')[1]
                float(valor_temperatura)
            except (ValueError, IndexError):
                log_file.write(f"{linha.strip()} | O valor da temperatura não é um float\n")
                linha_e_valida = False

        if linha_e_valida:
            linhas_totalmente_validas.append(linha)


# **02. Carregamento (Load)**

In [41]:
def agrupar_e_salvar_json(linhas_validas):
    dados_agrupados = {}
    for linha in linhas_validas:
        partes = linha.split('|')
        id_paciente = partes[0].split(':')[1]

        leitura = {
            "pressao": partes[1].split(':')[1],
            "saturacao": partes[2].split(':')[1],
            "temperatura": partes[3].split(':')[1]
        }

        if id_paciente not in dados_agrupados:
            dados_agrupados[id_paciente] = []

        dados_agrupados[id_paciente].append(leitura)

    nome_arquivo_json = 'pacientes_monitorados.json'
    with open(nome_arquivo_json, 'w', encoding='utf-8') as file:
        json.dump(dados_agrupados, file, indent=4)

    return nome_arquivo_json

agrupar_e_salvar_json(linhas_totalmente_validas)

'pacientes_monitorados.json'

# **03. Análise Preditiva e Geração de Relatório**

In [44]:
def gerar_relatorio(nome_arquivo_json):
    nome_arquivo_relatorio = 'relatorio_de_alertas.txt'
    pacientes_em_alerta = []

    with open(nome_arquivo_json, 'r', encoding='utf-8') as file:
          dados_pacientes = json.load(file)

    for id_paciente, leituras in dados_pacientes.items():
          alertas_do_paciente = []
          for leitura in leituras:

              # Para a Saturação
              try:
                  saturacao = int(leitura.get('saturacao'))
                  if saturacao < 92:
                      alertas_do_paciente.append(f"Saturação de {saturacao}% (abaixo do normal)")
              except (ValueError, TypeError):
                  pass

              # Para a Pressão Sistólica ou Diastólica
              try:
                  sistolica, diastolica = map(int, leitura.get('pressao').split('/'))
                  if sistolica > 180 or diastolica > 100:
                      alertas_do_paciente.append(f"Pressão de {sistolica}/{diastolica} (hipertensão)")
              except (ValueError, TypeError, AttributeError):
                  pass

              # Para a Temperatura
              try:
                  temperatura = float(leitura.get('temperatura'))
                  if temperatura >= 38.0:
                      alertas_do_paciente.append(f"Temperatura de {temperatura}ºC (febre)")
              except (ValueError, TypeError):
                  pass

          if alertas_do_paciente:
              pacientes_em_alerta.append({
                  "id": id_paciente,
                  "alertas": alertas_do_paciente
              })


    with open(nome_arquivo_relatorio, 'w', encoding='utf-8') as relatorio:
          if pacientes_em_alerta:
              relatorio.write("RELATÓRIO DE PACIENTES EM ESTADO DE ALERTA\n")

              for paciente in pacientes_em_alerta:
                  relatorio.write(f"Paciente ID: {paciente['id']}\n")
                  for alerta in paciente['alertas']:
                      relatorio.write(f"{alerta}\n")
                  relatorio.write("\n")
          else:
              relatorio.write("RELATÓRIO DE SAÚDE DIÁRIO\n")
              relatorio.write("Nenhum paciente está em estado de alerta hoje.\n")

gerar_relatorio('pacientes_monitorados.json')